In [44]:
# import essential spark dependencies

from pyspark.sql.functions import col, first, udf, explode, collect_set, countDistinct, lit
from pyspark.sql import SparkSession

In [2]:
# build spark session
spark = SparkSession.builder.appName("spotify").getOrCreate()

22/12/23 03:50:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [41]:
# reading as a DataFrame
df = spark.read\
          .option("multiline","true") \
          .json("../data/spotify/")

In [43]:
df.show()

+-------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+------+--------------------+--------------------+--------+--------------------+
|collaborative|         description|       external_urls| followers|                href|                  id|              images|                name|               owner|primary_color|public|         snapshot_id|              tracks|    type|                 uri|
+-------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+------+--------------------+--------------------+--------+--------------------+
|        false|Running Music Hit...|[https://open.spo...|[, 160152]|https://api.spoti...|5MLyX3xiNllq052Fs...|[[, https://i.scd...|Jogging Music 202...|[LoudKult, [https...|         null|  true|MTc1N

In [46]:
df.printSchema()

root
 |-- collaborative: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- external_urls: struct (nullable = true)
 |    |-- spotify: string (nullable = true)
 |-- followers: struct (nullable = true)
 |    |-- href: string (nullable = true)
 |    |-- total: long (nullable = true)
 |-- href: string (nullable = true)
 |-- id: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- height: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |    |    |-- width: string (nullable = true)
 |-- name: string (nullable = true)
 |-- owner: struct (nullable = true)
 |    |-- display_name: string (nullable = true)
 |    |-- external_urls: struct (nullable = true)
 |    |    |-- spotify: string (nullable = true)
 |    |-- href: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- uri: string (nullable = true)
 |-- primary_color

In [72]:
# fetch tracks info
# id is carried from the root to later join the table

tracks = df.select("id","tracks.*").select("id",explode(col("items")).alias("items")).select("id","items.*")


In [74]:
tracks.printSchema()

root
 |-- id: string (nullable = true)
 |-- added_at: timestamp (nullable = true)
 |-- added_by: struct (nullable = true)
 |    |-- external_urls: struct (nullable = true)
 |    |    |-- spotify: string (nullable = true)
 |    |-- href: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- uri: string (nullable = true)
 |-- is_local: boolean (nullable = true)
 |-- primary_color: string (nullable = true)
 |-- track: struct (nullable = true)
 |    |-- album: struct (nullable = true)
 |    |    |-- album_type: string (nullable = true)
 |    |    |-- artists: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- external_urls: struct (nullable = true)
 |    |    |    |    |    |-- spotify: string (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- name: string (nullable = true)
